# Stable Diffusion XL with ControlNet

## Install and Import Packages & Libraries

In [ ]:
# Install relevant modules and packages
%pip install --upgrade --quiet diffusers accelerate transformers opencv-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 MB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 67.9 MB/s eta 0:00:00


In [ ]:
# Import modules and packages
from diffusers import StableDiffusionXLPipeline, StableDiffusionXLImg2ImgPipeline, StableDiffusionXLControlNetXSPipeline, ControlNetXSAdapter, AutoencoderKL
import torch
from diffusers.utils import load_image
import numpy as np
import cv2
from PIL import Image, ImageDraw, ImageFont
import textwrap

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

seed = 88888
generator = torch.Generator(device).manual_seed(seed)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

/usr/local/lib/python3.10/dist-packages/diffusers/models/transformers/transformer_2d.py:34: FutureWarning: `Transformer2DModelOutput` is deprecated and will be removed in version 1.0.0. Importing `Transformer2DModelOutput` from `diffusers.models.transformer_2d` is deprecated and this will be removed in a future version. Please use `from diffusers.models.modeling_outputs import Transformer2DModelOutput`, instead.
  deprecate("Transformer2DModelOutput", "1.0.0", deprecation_message)


## Create Characters for the Comic

In [ ]:
# Create SDXL pipeline
pipe = StableDiffusionXLPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0", torch_dtype=torch.float16, variant="fp16", use_safetensors=True
    ).to(device)

# Define user prompts for characters
character_prompts = ["John F Kennedy, President of United States of America, a gentlemen with slick hair and a sharp suit; coloured cartoon comic"]
negative_prompt = "realistic, error, cropped, worst quality, low quality, jpeg artifacts, ugly, duplicate, morbid, mutilated, out of frame, extra fingers, mutated hands, poorly drawn hands, poorly drawn face, deformed, blurry, bad anatomy, bad proportions, extra limbs, cloned face, disfigured, gross proportions, malformed limbs, missing arms, missing legs, extra arms, extra legs, fused fingers, too many fingers, long neck, username, watermark, signature"

# Generate character images and save them
characters = []
for idx, prompt in enumerate(character_prompts):
    character_image = pipe(prompt,
                           num_inference_steps=50,
                           guidance_scale=7.5,
                           negative_prompt=negative_prompt,
                           generator=generator).images[0]
    character_image.save(f'character_{idx}.png')
    characters.append(character_image)

pipe.enable_model_cpu_offload()
torch.cuda.empty_cache()

model_index.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

Fetching 19 files:   0%|          | 0/19 [00:00<?, ?it/s]

model.fp16.safetensors:   0%|          | 0.00/1.39G [00:00<?, ?B/s]

model.fp16.safetensors:   0%|          | 0.00/246M [00:00<?, ?B/s]

scheduler/scheduler_config.json:   0%|          | 0.00/479 [00:00<?, ?B/s]

...

unet/config.json:   0%|          | 0.00/1.68k [00:00<?, ?B/s]

vae/config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,


## Create Comic Panel Based on Story Script and Character

In [ ]:
# Create prompts for each comic panel
story_script = [
    "President John F Kennedy in a dark blue suit sitting in his Oval office with (only 1 assistant) standing next to him briefing him on a report; coloured US cartoon comic.",
    "President John F Kennedy in a dark blue suit  having a discussion with his advisers on a long table in a meeting room; coloured US cartoon comic.",
    "President John F Kennedy in a dark blue suit talks to his advisers points to a paper report ; coloured US cartoon comic.",
    "President John F Kennedy's advisers on one side of table arguing angrily; coloured US cartoon comic.",
    "President John F Kennedy in a dark blue suit sitting in a leather sofa in the Oval office thinking hard.; coloured US cartoon comic.",
    "President John F Kennedy in a dark blue suit makes an announcement on a on podium to a room full of reporters.; coloured US cartoon comic."
    ]

In [ ]:
# Initialize the models and pipeline

controlnet_conditioning_scale = 0.5

vae = AutoencoderKL.from_pretrained("madebyollin/sdxl-vae-fp16-fix", torch_dtype=torch.float16)

controlnet = ControlNetXSAdapter.from_pretrained(
    "UmerHA/Testing-ConrolNetXS-SDXL-canny", torch_dtype=torch.float16)

pipe = StableDiffusionXLControlNetXSPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    vae = vae,
    controlnet=controlnet,
    torch_dtype=torch.float16).to(device)

pipe.enable_model_cpu_offload()
torch.cuda.empty_cache()

config.json:   0%|          | 0.00/631 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/737 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/85.0M [00:00<?, ?B/s]

Fetching 17 files:   0%|          | 0/17 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/2.78G [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/492M [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/10.3G [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [ ]:
# Download an image
image = characters[0]

# Get canny image
image = np.array(image)
image = cv2.Canny(image, 100, 200)
image = image[:, :, None]
image = np.concatenate([image, image, image], axis=2)
canny_image = Image.fromarray(image)

for idx, prompt in enumerate(story_script):
  # Generate images
  story_image = pipe(prompt,
                     controlnet_conditioning_scale=controlnet_conditioning_scale,
                     image=canny_image,
                     negative_prompt=negative_prompt,
                     generator=generator).images[0]
  story_image.save(f'story_{idx}.png')
  torch.cuda.empty_cache()

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

## Organize Images into Comic

In [ ]:
def create_comic_strip(images, texts, output_path, image_width, image_height, panels_horizontal, panels_vertical, border_size=10, text_height=150, font_path=None, font_size=30):
    """
    Organize images into a comic strip with borders and text boxes.

    Arguments:
    images: List of image file paths
    texts: List of text descriptions for each panel
    output_path: Path to save the final comic strip
    image_width: Width of each image
    image_height: Height of each image
    panels_horizontal: Number of panels placed horizontally
    panels_vertical: Number of panels placed vertically
    border_size: Size of the border around each panel
    text_height: Height of the text box at the bottom of each panel
    font_path: Path to the font file for the text
    font_size: Font size for the text
    """
    # Ensure the number of images matches the number of panels
    num_panels = panels_horizontal * panels_vertical
    if len(images) != num_panels:
        raise ValueError(f"The number of images ({len(images)}) does not match the total number of panels ({num_panels})")

    # Ensure the number of texts matches the number of panels
    if len(texts) != num_panels:
        raise ValueError(f"The number of texts ({len(texts)}) does not match the total number of panels ({num_panels})")

    # Calculate total dimensions of the comic strip
    total_width = (image_width + 2 * border_size) * panels_horizontal
    total_height = (image_height + 2 * border_size + text_height) * panels_vertical

    # Create a new image with the total dimensions
    new_image = Image.new('RGB', (total_width, total_height), 'white')

    # Load the font
    font = ImageFont.truetype(font_path, size=font_size) if font_path else ImageFont.load_default()

    # Paste images into the new image
    for i, (image_path, text) in enumerate(zip(images, texts)):
        image = Image.open(image_path)
        image = image.resize((image_width, image_height))

        # Create a panel with a border
        panel = Image.new('RGB', (image_width + 2 * border_size, image_height + 2 * border_size + text_height), 'white')
        panel.paste(image, (border_size, border_size))

        # Draw the text box
        draw = ImageDraw.Draw(panel)
        text_position = (border_size, image_height + 2 * border_size)
        draw.rectangle([text_position, (panel.width - border_size, panel.height - border_size)], fill="white")

        # Wrap the text to fit within the text box
        wrapped_text = textwrap.fill(text, width=(image_width // font_size * 2))

        # Add the text to the text box
        draw.text((text_position[0] + 10, text_position[1] + 10), wrapped_text, font=font, fill="black")

        # Calculate position in the new image
        x_offset = (i % panels_horizontal) * (image_width + 2 * border_size)
        y_offset = (i // panels_horizontal) * (image_height + 2 * border_size + text_height)
        new_image.paste(panel, (x_offset, y_offset))

    # Save the new comic strip image
    new_image.save(output_path)

In [ ]:
## EXAMPLE COMIC STRIP
images = ['/content/story_0.png', '/content/story_1.png',
          '/content/story_2.png', '/content/story_3.png',
          '/content/story_4.png', '/content/story_5.png']

texts = ["1. President John F Kennedy receives a report from his adviser that missile sites have been found in Cuba. The year was 1962.",
         "2. He called for a meeting with advisers to discuss what to do next. On one side he has the 'Doves' and on the other he has the 'Hawks'.",
         "3. President John F Kennedy: Gentlemen! The hour is upon us. Here is the report I received about Fidel Castro and the agreement with the Soviet Union to place nuclear missiles within attacking distance of the United States.",
         "4. The advisers argued about what should be done. Seven different options were laid on the table.",
         "5. President John F Kennedy thought to himself about what he should do next. The next move will change the course of history.",
         "6. After much deliberation, he set Robert Kennedy to strike a deal with Khrushchev. In the mean time, he announced to the world that a blockade will be introduced."
         ]

output_path = 'comic_strip.png'
image_width = 1024
image_height = 1024
panels_horizontal = 3
panels_vertical = 2
border_size = 10
text_height = 150
font_path = '/content/ComicNeue-BoldItalic.ttf'
font_size = 30

create_comic_strip(images, texts, output_path, image_width, image_height, panels_horizontal, panels_vertical, border_size, text_height, font_path, font_size)
